In [1]:
import json
import time
import requests
import hashlib
import csv
import pandas as pd
from datetime import datetime


C:\Users\danii\AppData\Local\Temp\ipykernel_24024\1837837796.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
class DomClickApi:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({"X-Service": "true",
                                    "Connection": "Keep-Alive",
                                     "User-Agent": "Android; 12; Google; google_pixel_5; 8.72.0; 8720006; ; NONAUTH"
                                     })

        # init (get cookies)
        self.get("https://api.domclick.ru/core/no-auth-zone/api/v1/ensure_session")
        self.get("https://ipoteka.domclick.ru/mobile/v1/feature_toggles")

    def get(self, url, **kwargs):
        self.__update_headers(url, **kwargs)
        result = self.session.get(url, **kwargs)
        #print(self.session.cookies.get_dict())
        return result

    def __update_headers(self, url, **kwargs):
        url = self.__get_prepared_url(url, **kwargs)
        sault = "ad65f331b02b90d868cbdd660d82aba0"
        timestamp = str(int(datetime.now().timestamp()))
        encoded = (sault + url + timestamp).encode("UTF-8")
        h = hashlib.md5(encoded).hexdigest()
        self.session.headers.update({"Timestamp": timestamp,
                                     "Hash": "v1:" + h,
                                     })

    def __get_prepared_url(self, url, **kwargs):
        p = requests.models.PreparedRequest()
        p.prepare(method="GET", url=url, **kwargs)
        return p.url



In [3]:

def pprint_json(json_str):
    try:
        json_object = json.loads(json_str)
        json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False).encode('utf8')
        print(json_formatted_str.decode())
    except:
        pass
        print(json_str)


In [4]:
def get_keys(d):
    if isinstance(d, dict):
        for k, v in d.items():
            yield k
            yield from list(get_keys(v))
    elif isinstance(d, list):
        for o in d:
            yield from list(get_keys(o))

In [10]:
remont = ['design', 'without_repair', 'standard']#, 'well_done'] #without_repair design standard well_done
balcons = [1, 2]
rooms = [1, 2, 3]
type_dome = ['loyout']#, 'flat'] #flat loyout
perec = ['panel', 'brick', 'panel']# 'monolith'] #monolith  brick panel block

offers_url = 'https://offers-service.domclick.ru/research/v5/offers/'
count_url = 'https://offers-service.domclick.ru/research/v5/offers/count/'

dca = DomClickApi()
for type in perec:
    for flat in rooms:
        for sd in type_dome:
            for rem in remont:
                for balcon in balcons:
                    res = dca.get(count_url, params={
                        "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",
                        "deal_type": "sale",
                        "category": "living",
                        "offer_type": ["flat", "layout"],
                        "rooms": flat,
                        'renovation': rem,
                        "wall_type": type,
                        "balconies": balcon
                    })
                    print("RES:", res)
                    print(res.text)
                    pprint_json(res.text)

                    count_obj = json.loads(res.text)
                    total = count_obj["pagination"]["total"]


RES: <Response [200]>
{"success":true,"pagination":{"offset":0,"total":72,"limit":30},"result":{}}
{
  "success": true,
  "pagination": {
    "offset": 0,
    "total": 72,
    "limit": 30
  },
  "result": {}
}
RES: <Response [200]>
{"success":true,"pagination":{"offset":0,"total":6,"limit":30},"result":{}}
{
  "success": true,
  "pagination": {
    "offset": 0,
    "total": 6,
    "limit": 30
  },
  "result": {}
}
RES: <Response [200]>
{"success":true,"pagination":{"offset":0,"total":93,"limit":30},"result":{}}
{
  "success": true,
  "pagination": {
    "offset": 0,
    "total": 93,
    "limit": 30
  },
  "result": {}
}
RES: <Response [200]>
{"success":true,"pagination":{"offset":0,"total":6,"limit":30},"result":{}}
{
  "success": true,
  "pagination": {
    "offset": 0,
    "total": 6,
    "limit": 30
  },
  "result": {}
}
RES: <Response [200]>
{"success":true,"pagination":{"offset":0,"total":496,"limit":30},"result":{}}
{
  "success": true,
  "pagination": {
    "offset": 0,
    "tot

In [ ]:
#def get_seo_info(J_data):
#    for j in J_data['items']:
#        
   

In [11]:
offset = 0
df_data = []
for type in perec:
    for flat in rooms:
        for sd in type_dome:
            for rem in remont:
                for balcon in balcons:
                    for offset in range(0, 700, 10):
                        try:
                            res = dca.get(offers_url, params={
                                        "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",
                                        "deal_type": "sale",
                                        "category": "living",
                                        "offer_type": ["flat", "layout"],
                                        "offset": offset,
                                        "limit": 10,
                                        "rooms": flat,
                                        "wall_type": type,
                                        "renovation": rem,
                                        'balconies': balcon
                            })
                            print("RES:", res)
                            #pprint_json(res.text)
                            offers_obj = json.loads(res.text)
                            #print(list(get_keys(offers_obj)))
                            total = offers_obj["pagination"]["total"]
                            result_data = offers_obj['result']#['items']
                            items = result_data['items']
                            for item in items:
                                address = item['address']
                                price = item['price_info']
                                house = item['house']
                                object_info = item['object_info']
                                        #print(offers_obj)
                                        #renovation = item['renovation']
                                row = {
                                                #'address_id': address['id'],
                                    'address_name': address['name'],
                                    'price': price['price'],
                                    'square_price': price['square_price'],
                                    'floors': object_info['floor'],
                                    'total_floors': house['floors'],
                                #'chat_avail': item['chat_available'],
                                    'rooms': object_info['rooms'],
                                    'area': object_info['area'],
                                    'local': address['locality']['name'],
                                #'type_house': type,
                                    'type_home': sd,
                                    'remont': rem,
                                    'balcon': balcon,
                                #'year': result_data['items'][0]

                                    }
                                            # Находим seo_uri для данной станции метро
                                            #for seo_subway in seo_info['subways']:
                                            #    if seo_subway['guid'] == subway['guid']:
                                            #        row['seo_uri'] = seo_subway['seo_uri']
                                            #        break
                                        # Добавляем словарь в список данных
                                df_data.append(row)
    #                           
                        except ConnectionError:
                            print("error")
df = pd.DataFrame(df_data)
df.to_csv('12.csv')
                            
#df = df.drop_duplicates(subset=['address_name'])
                            


RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>
RES: <Response [200]>


KeyboardInterrupt: 

In [9]:
m = pd.read_csv('12.csv')
m

,Unnamed: 0,address_name,price,square_price,floors,total_floors,rooms,area,local,type_home,remont,balcon
0,0,"улица Липчанского, 6",10400000,266666,15,17,1,39.0,Москва,loyout,design,1
1,1,"улица Маршала Голованова, 13",11900000,316489,8,17,1,37.6,Москва,loyout,design,1
2,2,"улица Введенского, 20к1",13300000,380000,9,9,1,35.0,Москва,loyout,design,1
3,3,"Варшавское шоссе, 50",12799000,365685,12,12,1,35.0,Москва,loyout,design,1
4,4,"Профсоюзная улица, 85к1",13800000,431250,4,9,1,32.0,Москва,loyout,design,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11755,11755,"Новоясеневский проспект, 19к1",16000000,274914,3,9,3,58.2,Москва,flat,well_done,2
11756,11756,"улица Коминтерна, 12",25500000,279299,17,17,3,91.3,Москва,flat,well_done,2
11757,11757,"Зеленоград, к829",21000000,267175,5,19,3,78.6,Зеленоград,flat,well_done,2
11758,11758,"шоссе Энтузиастов, 11Ак4",35500000,473333,3,17,3,75.0,Москва,flat,well_done,2


In [181]:
df = pd.read_csv('data/1_data.csv')
df['d'] = df['type_home'].replace(['loyout', 'flat'], [0,1])
df

C:\Users\danii\AppData\Local\Temp\ipykernel_18576\2795032174.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['d'] = df['type_home'].replace(['loyout', 'flat'], [0,1])


,Unnamed: 0,address_name,price,square_price,floors,rooms,area,local,type_house,type_home,remont,total_floors,balcon,d
0,0,"улица Грекова, 3к2",22500000,350467,16,3,64.20,Москва,panel,loyout,design,16,2,0
1,1,"Береговой проезд, 9к5",17600000,451282,12,1,39.00,Москва,panel,loyout,design,12,2,0
2,2,"Сивашская улица, 7",36500000,325892,17,4,112.00,Москва,panel,loyout,design,17,2,0
3,3,"Окская улица, 5к1",44999999,409090,23,4,110.00,Москва,panel,loyout,design,23,2,0
4,4,"Дубнинская улица, 2к1",16499000,260812,12,3,63.26,Москва,panel,loyout,design,12,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35470,35470,"Детский проезд, 1к1",6400000,118518,19,2,54.00,Казань,monolith,flat,standard,19,1,1
35471,35471,"Нижняя улица, 8",7800000,95823,17,3,81.40,Казань,monolith,flat,standard,17,1,1
35472,35472,"улица Рауиса Гареева, 109",5650000,200996,23,0,28.11,Казань,monolith,flat,standard,23,1,1
35473,35473,"улица Нурихана Фаттаха, 17/4",6000000,92307,18,2,65.00,Казань,monolith,flat,standard,18,1,1


In [183]:
cor = df['price'].corr(df['d'])
cor

-0.11211767773158297

In [5]:
main = pd.read_csv('data/1_data.csv')
#main = pd.concat([df, main], ignore_index=True)
#main.to_csv('1_data.csv')

In [737]:
#main = main.drop_duplicates(subset=['address_name'])

In [14]:
main = main.drop(['Unnamed: 0.58'], axis = 1, )

In [16]:

main.to_csv('1_data.csv')


In [17]:
main

,address_name,price,square_price,floors,rooms,area,local,type_house,type_home,remont,total_floors,balcon
0,"улица Грекова, 3к2",22500000,350467,16,3,64.20,Москва,panel,loyout,design,16,2
1,"Береговой проезд, 9к5",17600000,451282,12,1,39.00,Москва,panel,loyout,design,12,2
2,"Сивашская улица, 7",36500000,325892,17,4,112.00,Москва,panel,loyout,design,17,2
3,"Окская улица, 5к1",44999999,409090,23,4,110.00,Москва,panel,loyout,design,23,2
4,"Дубнинская улица, 2к1",16499000,260812,12,3,63.26,Москва,panel,loyout,design,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...
35470,"Детский проезд, 1к1",6400000,118518,19,2,54.00,Казань,monolith,flat,standard,19,1
35471,"Нижняя улица, 8",7800000,95823,17,3,81.40,Казань,monolith,flat,standard,17,1
35472,"улица Рауиса Гареева, 109",5650000,200996,23,0,28.11,Казань,monolith,flat,standard,23,1
35473,"улица Нурихана Фаттаха, 17/4",6000000,92307,18,2,65.00,Казань,monolith,flat,standard,18,1
